In [ ]:
import numpy as np
import pandas as pd

In [ ]:
gcounts = dict()
with open('../data/gifts.csv') as f:
    f.readline()
    for line in f: 
        if '_' in line:
            line = line.strip().split('_')
            gtype = line[0]
            gnum = int(line[1])
            current_count = gcounts.get(gtype, 0)
            assert (gnum == current_count)
            gcounts[gtype] = current_count + 1

In [ ]:
gcounts

In [ ]:
dist = dict()
dist['horse'] = max(0, np.random.normal(5,2,1)[0])
dist['ball'] = max(0, 1 + np.random.normal(1,0.3,1)[0])
dist['bike'] = max(0, np.random.normal(20,10,1)[0])
dist['train'] = max(0, np.random.normal(10,5,1)[0])
dist['coal'] = 47 * np.random.beta(0.5,0.5,1)[0]
dist['book'] = np.random.chisquare(2,1)[0]
dist['doll'] = np.random.gamma(5,1,1)[0]
dist['blocks'] = np.random.triangular(5,10,20,1)[0]
dist['gloves'] = 3.0 + np.random.rand(1)[0] if np.random.rand(1) < 0.3 else np.random.rand(1)[0]

In [ ]:
def get_weights(confidence):
    pass

In [ ]:
# temporarily
gweights = dist

In [ ]:
from gurobipy import Model, GRB, quicksum, LinExpr
model = Model('packing')
model.params.UpdateMode = 1
items = list(gweights.keys())

# find the upper bounds
ubs = dict()
for item, weight in gweights.items():
    ubs[item] = 50 // weight
    
vars = dict()
# define the variables
for i in range(1000):
    for item in items:
        vars[(i, item)] = model.addVar(lb=0.0, ub=ubs[item], vtype=GRB.INTEGER)
        
# constraint: at lest three items in each bag
for i in range(1000):
    model.addConstr(quicksum([vars[(i, item)] for item in items]) >= 3)

# constraint: weight of each bag less than 50
for i in range(1000):
    model.addConstr(LinExpr([gweights[item] for item in items], [vars[(i, item)] for item in items]) <= 50)
    
# constraint: no more gifts packed than available
for item in items:
    model.addConstr(quicksum([vars[(i, item)] for i in range(1000)]) <= gcounts[item])

# objective function
model.setObjective(LinExpr([gweights[item] for item in items for i in range(1000)],
                           [vars[(i, item)] for item in items for i in range(1000)]), GRB.MAXIMIZE)

model.update()
model.optimize()